In [ ]:
from tensorflow.keras.layers import Conv2D, Add, BatchNormalization, DepthwiseConv2D, GlobalAveragePooling2D, concatenate
from tensorflow.keras.layers import Input, Dense, Dropout, UpSampling2D, Flatten, Activation, Layer, AveragePooling2D
from tensorflow.python.keras.layers.convolutional import Conv2DTranspose
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.models import Model, load_model
from google.colab import drive
import tensorflow as tf
import numpy as np
import warnings
import os

#drive.mount('/content/drive')
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Encoder

## Squezee and exitation

In [ ]:
#######################################
######### Squeeze Excitation ##########
#######################################

class SEBlock(Layer):
  def __init__(self, t, salida):
    super(SEBlock, self).__init__()
    mid_c= salida // t
    self.pool= AveragePooling2D(1)
    self.bat1= BatchNormalization()
    self.bat2= BatchNormalization()
    self.conv1= Conv2D(mid_c, 1, padding= 'same', activation= 'relu')
    self.conv2= Conv2D(salida, 1, padding= 'same', activation= 'sigmoid')
  
  def call(self, entrada):
    x= self.pool(entrada)
    x= self.conv1(x)
    x= self.bat1(x)
    x= self.conv2(x)
    x= self.bat2(x)
    x= entrada* x

    return x

## MBConv

In [ ]:
#######################################
############### MBConv ################
#######################################

class expancion(Layer):
  def __init__(self, t, expand= 64):
    super(expancion, self).__init__()

    self.conv1= Conv2D(expand* t, kernel_size= 1, padding= 'same')
    self.batch1= BatchNormalization()
    self.act= Activation('relu6')

  def call(self, entrada):
    x= self.conv1(entrada)
    x= self.batch1(x)
    return self.act(x)


class depthwise(Layer):
  def __init__(self, stride= 2):
    super(depthwise, self).__init__()

    self.depth= DepthwiseConv2D(kernel_size= 3,
                                strides= stride,
                                padding= 'same')
    self.batch1= BatchNormalization()
    self.act= Activation('relu6')

  def call(self, entrada):
    x= self.depth(entrada)
    x= self.batch1(x)
    return self.act(x)


class proyeccion(Layer):
  def __init__(self, salida):
    super(proyeccion, self).__init__()
    self.conv1= Conv2D(salida, kernel_size= 1, padding= 'same')
    self.batch1= BatchNormalization()
    self.act= Activation('relu6')

  def call(self, entrada):
    x= self.conv1(entrada)
    x= self.batch1(x)
    return self.act(x)
    

class bottleneck(Layer):
  def __init__(self, t, expand, salida, stride):
    super(bottleneck, self).__init__()
    self.exp= expancion(t, expand)
    self.dept= depthwise(stride)
    self.se= SEBlock(t, t* expand)
    self.pro= proyeccion(salida)

  def call(self, entrada):
    x= self.exp(entrada)
    x= self.dept(x)
    x= self.se(x)
    x= self.pro(x)
    if x.shape[-1]== entrada.shape[-1]:
      x= Add()([entrada, x])
    return x

## Fusion MBConv

In [ ]:
#######################################
############### MBFusion ##############
#######################################

class F_expansion(Layer):
  def __init__(self, t, expand, stride):
    super(F_expansion, self).__init__()
    self.conv= Conv2D(expand* t, kernel_size= 3, 
                      strides= stride,
                      padding= 'same')
    self.batch= BatchNormalization()
    self.act= Activation('relu6')

  def call(self, entrada):
    x= self.conv(entrada)
    x= self.batch(x)
    return self.act(x)


class F_proyeccion(Layer):
  def __init__(self, salida, stride):
    super(F_proyeccion, self).__init__()
    self.conv= Conv2D(salida, kernel_size= 1, 
                      strides= stride,
                      padding= 'same')
    self.batch= BatchNormalization()
    self.act= Activation('relu6')

  def call(self, entrada):
    x= self.conv(entrada)
    x= self.batch(x)
    return self.act(x)


class F_Bottleneck(Layer):
  def __init__(self, t, expand, salida, stride):
    super(F_Bottleneck, self).__init__()
    self.f_exp= F_expansion(t, expand, stride)
    self.f_se= SEBlock(t, t* expand)
    self.f_pro= F_proyeccion(salida, stride)

  def call(self, entrada):
    x= self.f_exp(entrada)
    x= self.f_se(x)
    x= self.f_pro(x)

    if x.shape[-1]== entrada.shape[-1]:
      x= Add()([entrada, x])
    return x

# Decoder

In [ ]:
class up_conc(Layer):
  def __init__(self, ups= 2):
    super(up_conc, self).__init__()
    self.up_sampling= UpSampling2D(ups)

  def call(self, entrada, res):
    x= self.up_sampling(entrada)
    return concatenate([x, res])

class dec_expancion(Layer):
  def __init__(self, t, expand= 64):
    super(dec_expancion, self).__init__()

    self.conv1= Conv2D(expand* t, kernel_size= 1,
                       padding= 'same',
                       activation= 'relu6')
    self.batch1= BatchNormalization()

  def call(self, entrada):
    x= self.conv1(entrada)
    return self.batch1(x)


class dec_depthwise(Layer):
  def __init__(self, stride= 2):
    super(dec_depthwise, self).__init__()

    self.depth= DepthwiseConv2D(kernel_size= 3,
                                padding= 'same',
                                activation= 'relu6')
    self.batch1= BatchNormalization()

  def call(self, entrada):
    x= self.depth(entrada)
    return self.batch1(x)
    


class dec_proyeccion(Layer):
  def __init__(self, salida, transp, st):
    super(dec_proyeccion, self).__init__()
    self.st= st
    if self.st== 1:
      self.trans= Conv2DTranspose(transp, 3, padding= 'same',
                                  activation= 'relu6')
    self.conv1= Conv2D(salida, kernel_size= 1, 
                       padding= 'same',
                       activation= 'relu6')
    self.batch1= BatchNormalization()

  def call(self, entrada):
    x= self.conv1(entrada)
    if self.st== 1:
      x= self.batch1(x)
      return self.trans(x)
    else:
      return self.batch1(x)
    

class dec_bottleneck(Layer):
  def __init__(self, t, expand, salida, transp= 0, st= 0):
    super(dec_bottleneck, self).__init__()
    self.exp= dec_expancion(t, expand)
    self.dept= dec_depthwise()
    self.se= SEBlock(t, t* expand)
    self.pro= dec_proyeccion(salida, transp, st)

  def call(self, entrada):
    x= self.exp(entrada)
    x= self.dept(x)
    x= self.se(x)
    x= self.pro(x)
    if x.shape[-1]== entrada.shape[-1]:
      x= Add()([entrada, x])
    return x

# Generación del modelo

In [ ]:
class EfficientV2(Model):
  def __init__(self, num_clases):
    super(EfficientV2, self).__init__()

    self.conv= Conv2D(24, 3, strides= 2, padding= "same", activation= 'relu6')
    self.batch= BatchNormalization()

    self.Fused_MBC1_1= F_Bottleneck(1, 24, 24, 1)
    self.Fused_MBC1_2= F_Bottleneck(1, 24, 24, 1)

    self.Fused_MBC2_1= F_Bottleneck(4, 24, 48, 2)
    self.Fused_MBC2_2= F_Bottleneck(4, 24, 48, 1)
    self.Fused_MBC2_3= F_Bottleneck(4, 24, 48, 1)
    self.Fused_MBC2_4= F_Bottleneck(4, 24, 48, 1)

    self.Fused_MBC3_1= F_Bottleneck(4, 48, 64, 2)
    self.Fused_MBC3_2= F_Bottleneck(4, 48, 64, 1)
    self.Fused_MBC3_3= F_Bottleneck(4, 48, 64, 1)
    self.Fused_MBC3_4= F_Bottleneck(4, 48, 64, 1)

    self.MBC1_1= bottleneck(4, 64, 128, 2)
    self.MBC1_2= bottleneck(4, 64, 128, 1)
    self.MBC1_3= bottleneck(4, 64, 128, 1)
    self.MBC1_4= bottleneck(4, 64, 128, 1)
    self.MBC1_5= bottleneck(4, 64, 128, 1)
    self.MBC1_6= bottleneck(4, 64, 128, 1)

    self.MBC2_1= bottleneck(6, 128, 160, 1)
    self.MBC2_2= bottleneck(6, 128, 160, 1)
    self.MBC2_3= bottleneck(6, 128, 160, 1)
    self.MBC2_4= bottleneck(6, 128, 160, 1)
    self.MBC2_5= bottleneck(6, 128, 160, 1)
    self.MBC2_6= bottleneck(6, 128, 160, 1)
    self.MBC2_7= bottleneck(6, 128, 160, 1)
    self.MBC2_8= bottleneck(6, 128, 160, 1)
    self.MBC2_9= bottleneck(6, 128, 160, 1)

    self.MBC3_1= bottleneck(6, 160, 256, 2)
    self.MBC3_2= bottleneck(6, 160, 256, 1)
    self.MBC3_3= bottleneck(6, 160, 256, 1)
    self.MBC3_4= bottleneck(6, 160, 256, 1)
    self.MBC3_5= bottleneck(6, 160, 256, 1)
    self.MBC3_6= bottleneck(6, 160, 256, 1)
    self.MBC3_7= bottleneck(6, 160, 256, 1)
    self.MBC3_8= bottleneck(6, 160, 256, 1)
    self.MBC3_9= bottleneck(6, 160, 256, 1)
    self.MBC3_10= bottleneck(6, 160, 256, 1)
    self.MBC3_11= bottleneck(6, 160, 256, 1)
    self.MBC3_12= bottleneck(6, 160, 256, 1)
    self.MBC3_13= bottleneck(6, 160, 256, 1)
    self.MBC3_14= bottleneck(6, 160, 256, 1)
    self.MBC3_15= bottleneck(6, 160, 256, 1)

    self.conv1= Conv2D(1280, 1, padding= 'same', activation= 'relu6')
    self.batch1= BatchNormalization()
    self.convt= Conv2DTranspose(256, 3, padding= 'same', activation= 'relu6')
    self.batch2= BatchNormalization()

    self.upc1= up_conc(1)
    self.upc2= up_conc()

    self.dec_MBC3_1= dec_bottleneck(6, 160, 256)
    self.dec_MBC3_2= dec_bottleneck(6, 160, 256)
    self.dec_MBC3_3= dec_bottleneck(6, 160, 256)
    self.dec_MBC3_4= dec_bottleneck(6, 160, 256)
    self.dec_MBC3_5= dec_bottleneck(6, 160, 256)
    self.dec_MBC3_6= dec_bottleneck(6, 160, 256)
    self.dec_MBC3_7= dec_bottleneck(6, 160, 256)
    self.dec_MBC3_8= dec_bottleneck(6, 160, 256)
    self.dec_MBC3_9= dec_bottleneck(6, 160, 256)
    self.dec_MBC3_10= dec_bottleneck(6, 160, 256)
    self.dec_MBC3_11= dec_bottleneck(6, 160, 256)
    self.dec_MBC3_12= dec_bottleneck(6, 160, 256)
    self.dec_MBC3_13= dec_bottleneck(6, 160, 256)
    self.dec_MBC3_14= dec_bottleneck(6, 160, 256)
    self.dec_MBC3_15= dec_bottleneck(6, 160, 256, 160, 1)

  def call(self, entrada_tensor):
    x= self.conv(entrada_tensor)
    x= self.batch(x)

    x= self.Fused_MBC1_1(x)
    x= self.Fused_MBC1_2(x)

    x= self.Fused_MBC2_1(x)
    x= self.Fused_MBC2_2(x)
    x= self.Fused_MBC2_3(x)
    x= self.Fused_MBC2_4(x)

    x= self.Fused_MBC3_1(x)
    x= self.Fused_MBC3_2(x)
    x= self.Fused_MBC3_3(x)
    x= self.Fused_MBC3_4(x)

    x= self.MBC1_1(x)
    x= self.MBC1_2(x)
    x= self.MBC1_3(x)
    x= self.MBC1_4(x)
    x= self.MBC1_5(x)
    x= self.MBC1_6(x)

    x= self.MBC2_1(x)
    x= self.MBC2_2(x)
    x= self.MBC2_3(x)
    x= self.MBC2_4(x)
    x= self.MBC2_5(x)
    x= self.MBC2_6(x)
    x= self.MBC2_7(x)
    x= self.MBC2_8(x)
    x= self.MBC2_9(x)

    x= self.MBC3_1(x)
    x= self.MBC3_2(x)
    x= self.MBC3_3(x)
    x= self.MBC3_4(x)
    x= self.MBC3_5(x)
    x= self.MBC3_6(x)
    x= self.MBC3_7(x)
    x= self.MBC3_8(x)
    x= self.MBC3_9(x)
    x= self.MBC3_10(x)
    x= self.MBC3_11(x)
    x= self.MBC3_12(x)
    x= self.MBC3_13(x)
    x= self.MBC3_14(x)
    x= self.MBC3_15(x)
    res4= x

    x= self.conv1(x)
    x= self.batch1(x)

    # Deocder
    x= self.convt(x)
    x= self.batch2(x)

    x= self.upc1(x, res4)
    x= self.dec_MBC3_1(x)
    x= self.dec_MBC3_2(x)
    x= self.dec_MBC3_3(x)
    x= self.dec_MBC3_4(x)
    x= self.dec_MBC3_5(x)
    x= self.dec_MBC3_6(x)
    x= self.dec_MBC3_7(x)
    x= self.dec_MBC3_8(x)
    x= self.dec_MBC3_9(x)
    x= self.dec_MBC3_10(x)
    x= self.dec_MBC3_11(x)
    x= self.dec_MBC3_12(x)
    x= self.dec_MBC3_13(x)
    x= self.dec_MBC3_14(x)
    x= self.dec_MBC3_15(x)



    return x

  def build_graph(self, raw_shape):
    x= tf.keras.layers.Input(shape= raw_shape)
    return tf.keras.Model(inputs=[x], outputs= self.call(x))

In [ ]:
raw_input = (160, 160, 3)

mi_modelo= EfficientV2(2)
y= mi_modelo(tf.ones(shape=(0, *raw_input)))

mi_modelo.build_graph(raw_input).summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_1695 (Conv2D)           (None, 80, 80, 24)   672         ['input_6[0][0]']                
                                                                                                  
 batch_normalization_2023 (Batc  (None, 80, 80, 24)  96          ['conv2d_1695[0][0]']            
 hNormalization)                                                                                  
                                                                                            